In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

In [0]:
dbutils.widgets.text("container", "")
container=dbutils.widgets.get("container")
dbutils.widgets.text("blob", "")
blob=dbutils.widgets.get("blob")
dbutils.widgets.text("delta_path", "")
container=dbutils.widgets.get("delta_path")

In [0]:
%python
gen2name = dbutils.secrets.get(scope="scope", key="adlsname")
genkey = dbutils.secrets.get(scope="scope", key="adlskey")
bame = dbutils.secrets.get(scope="scope", key="blobname")
blokey = dbutils.secrets.get(scope="scope", key="blobkey")

In [0]:
spark_config=f"fs.azure.account.key.{gen2name}.dfs.core.windows.net"

In [0]:

display(dbutils.fs.ls('/'))


In [0]:
# Remove or comment out any lines like the following if they exist:
# spark.conf.set("spark.hadoop.fs.azure.account.key.newsadls129.dfs.core.windows.net", "genkey")


In [0]:
%python
spark.conf.set("fs.azure.account.auth.type.newsadls129.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.newsadls129.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.newsadls129.dfs.core.windows.net", "80e80bec-2a7e-443b-ba50-d342e90e77e0")
spark.conf.set("fs.azure.account.oauth2.client.secret.newsadls129.dfs.core.windows.net", "7850974d-53d0-4462-a984-602ae1890c06")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.newsadls129.dfs.core.windows.net", "https://login.microsoftonline.com/441dc366-2988-4558-8827-718893181569/oauth2/token")

In [0]:
spark.conf.set("spark.hadoop.fs.azure.account.auth.type.newsadls129.dfs.core.windows.net", "OAuth")
spark.conf.set("spark.hadoop.fs.azure.account.oauth.provider.type.newsadls129.dfs.core.windows.net", "AzureAD")
spark.conf.set("spark.hadoop.fs.azure.account.oauth2.client.id.newsadls129.dfs.core.windows.net", "80e80bec-2a7e-443b-ba50-d342e90e77e0")
spark.conf.set("spark.hadoop.fs.azure.account.oauth2.client.secret.newsadls129.dfs.core.windows.net", "7850974d-53d0-4462-a984-602ae1890c06")
spark.conf.set("spark.hadoop.fs.azure.account.oauth2.client.endpoint.newsadls129.dfs.core.windows.net", "https://login.microsoftonline.com/441dc366-2988-4558-8827-718893181569/oauth2/v2.0/token")


In [0]:
%python
spark.conf.set(
    "fs.azure.account.key.newsadls129.dfs.core.windows.net",
    dbutils.secrets.get(
        scope="scope",
        key="adlskey"
    )
)

In [0]:

spark.conf.set(
    spark_config, 
    genkey)

In [0]:
%python
delta_path = ""
inputpath = f"abfss://input@{bame}.dfs.core.windows.net/input"
outputpath = f"abfss://sales@{gen2name}.dfs.core.windows.net/output"

In [0]:
%python
# Use the abfss driver to access Azure storage securely
df_source = spark.read.format("csv") \
    .option("header", "true") \
    .load("abfss://sales@newsadls129.dfs.core.windows.net/output")

In [0]:
df_source=df_source.show()

In [0]:
%run "/Workspace/Repos/91madhu12@gmail.com/bricksrepos/levelschema"


In [0]:
%python
file_name = 'level_details2'
if file_name == 'level_details2':
    print("inside if")
    table_name = 'level_details2'
    schema_name = table_name + "_schema"


In [0]:
print(type(df_source))


In [0]:
%python
# Load the DataFrame from the table or source
df_source = spark.table("datalevel_db.level")

# Write the DataFrame to the Delta table
df_source.write.format("delta").mode("append").saveAsTable("datalevel_db.level")

In [0]:
insert_count = 0  # Correct variable name
myerror = 'concurrent'

def run_until(query_insert):
    global insert_count, myerror
    flag = True
    while flag and myerror == 'concurrent':  # Corrected condition
        try:
            insert_count += 1  # Corrected variable name
            print("Try count:", insert_count)
            # Add your `query_insert` execution here
            flag = False  # Exit loop if successful
        except Exception as e:  # Corrected syntax
            if "concurrent" in str(e).lower():
                print("Inside if: Concurrent issue detected")
                myerror = "concurrent"
            else:
                print("Inside else: Other error occurred")
                myerror = str(e)  # Store the actual error
                flag = False  # Exit loop on non-concurrent errors


In [0]:
%sql
select * from datalevel_db.level